In [1]:
from models.TernausNet import *

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os
import time

In [2]:
images = np.load("/media/bot/DANE/MED_DATASETS/NARZ_ZAGRO2/FILES/imgs16_half.npy")
labels = np.load("/media/bot/DANE/MED_DATASETS/NARZ_ZAGRO2/FILES/masks16_half.npy")


N = images.shape[0]
W = images.shape[1]
H = images.shape[2]
C = labels.shape[3]

In [3]:
x_train, x_test_valid, y_train, y_test_valid = train_test_split(images, labels, test_size=0.4, random_state=4)
x_valid, x_test, y_valid, y_test = train_test_split(x_test_valid, y_test_valid, test_size=0.5, random_state=4)

print(x_train.shape, y_train.shape)

(1020, 256, 256, 1) (1020, 256, 256, 5)


In [4]:
seg_model = TernausNet(img_shape = x_train[0].shape, num_of_class = 5, learning_rate = 2e-4, path = "saved_models/UNet_test")
seg_model.show_model()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 64) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           activation[0][0]                 
______________________________________________________________________________________________

In [ ]:
history = seg_model.train(x_train, y_train, x_valid, y_valid, epoch = 100, batch_size = 8)

Epoch 1/100
128/128 [==============================] - 1038s 8s/step - loss: 12.8524 - dice_coef_multilabel: 0.0408 - iou_multilabel: 0.0236 - val_loss: 0.6342 - val_dice_coef_multilabel: 0.2014 - val_iou_multilabel: 0.1297
INFO:tensorflow:Assets written to: saved_models/UNet_test/assets
Epoch 2/100
128/128 [==============================] - 983s 8s/step - loss: 0.4996 - dice_coef_multilabel: 0.2903 - iou_multilabel: 0.2214 - val_loss: 0.2769 - val_dice_coef_multilabel: 0.4184 - val_iou_multilabel: 0.3620
INFO:tensorflow:Assets written to: saved_models/UNet_test/assets
Epoch 3/100
128/128 [==============================] - 808s 6s/step - loss: 0.2182 - dice_coef_multilabel: 0.4558 - iou_multilabel: 0.3986 - val_loss: 0.1489 - val_dice_coef_multilabel: 0.4528 - val_iou_multilabel: 0.3968
INFO:tensorflow:Assets written to: saved_models/UNet_test/assets
Epoch 4/100
128/128 [==============================] - 850s 7s/step - loss: 0.1400 - dice_coef_multilabel: 0.4973 - iou_multilabel: 0.437

INFO:tensorflow:Assets written to: saved_models/UNet_test/assets
Epoch 32/100
128/128 [==============================] - 924s 7s/step - loss: 0.0094 - dice_coef_multilabel: 0.8873 - iou_multilabel: 0.8278 - val_loss: 0.0141 - val_dice_coef_multilabel: 0.8427 - val_iou_multilabel: 0.7706
Epoch 33/100
128/128 [==============================] - 772s 6s/step - loss: 0.0074 - dice_coef_multilabel: 0.9024 - iou_multilabel: 0.8466 - val_loss: 0.0121 - val_dice_coef_multilabel: 0.8542 - val_iou_multilabel: 0.7867
INFO:tensorflow:Assets written to: saved_models/UNet_test/assets
Epoch 34/100
128/128 [==============================] - 771s 6s/step - loss: 0.0075 - dice_coef_multilabel: 0.9010 - iou_multilabel: 0.8419 - val_loss: 0.0109 - val_dice_coef_multilabel: 0.8607 - val_iou_multilabel: 0.7958
INFO:tensorflow:Assets written to: saved_models/UNet_test/assets
Epoch 35/100
128/128 [==============================] - 770s 6s/step - loss: 0.0061 - dice_coef_multilabel: 0.8890 - iou_multilabel: 0.8